# 1. IMAGE PREPROCESSING 

## Importing Required Libraries

In [ ]:
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
import math
import tensorflow as tf

## Creating a Folder for Preprocessed Images

In [ ]:
os.makedirs("preprocessed_images", exist_ok=True)

## Loop Through Each Image

In [ ]:
for filename in os.listdir("original_images"):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.JPG', '.webp')):
        img_path = os.path.join("original_images", filename)

## Opening and Converting Each Image to RGB Format

In [ ]:
        img = Image.open(img_path).convert("RGB")

## Resizing the Image to 224x224 Pixels

In [ ]:
        img = img.resize((224, 224))

## Normalizing the Pixel Values Between 0 and 1

In [ ]:
        img_array = np.array(img) / 255.0
        img = Image.fromarray((img_array * 255).astype('uint8'))

## Saving the Preprocessed Image

In [ ]:
        img.save(os.path.join("preprocessed_images", filename))

## Displaying Preprocessed Images

### Loading preprocessed images

In [ ]:
preprocessed_images_path = "preprocessed_images"
image_files = [f for f in os.listdir(preprocessed_images_path) if f.endswith(('.jpg', '.jpeg', '.png', '.JPG', '.webp'))]

### Calculating grid size

In [ ]:
num_images = len(image_files)
cols = 5  # Number of columns
rows = math.ceil(num_images / cols)  # Calculating number of rows needed

### Displaying images

In [ ]:
plt.figure(figsize=(cols * 2, rows * 2))  
plt.suptitle("Preprocessed Images", fontsize=16) 

for i, image_file in enumerate(image_files):
    img_path = os.path.join(preprocessed_images_path, image_file)
    img = Image.open(img_path)

    plt.subplot(rows, cols, i + 1)  
    plt.imshow(img)
    plt.axis('off')
    
    image_name = os.path.splitext(image_file)[0] 
    plt.title(image_name)

plt.tight_layout()
plt.subplots_adjust(top=0.94)
plt.show()

# 2. IMAGE AUGMENTATION

## Creating a Folder for Augmented Images

In [ ]:
os.makedirs("augmented_images", exist_ok=True)

## Configuring Data Augmentation Settings

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.5, 1.5],
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
)

## Loop Through Each Preprocessed Image

In [ ]:
for filename in os.listdir("preprocessed_images"):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.JPG', '.webp')):
        img_path = os.path.join("preprocessed_images", filename)

## Loading the Image and Converting It to an Array

In [ ]:
        img = tf.keras.preprocessing.image.load_img(img_path)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = np.expand_dims(img_array, 0)

## Generating and Saving 5 Augmented Versions of Each Image

In [ ]:
        i = 0
        for batch in datagen.flow(img_array, batch_size=1):
            aug_img = tf.keras.preprocessing.image.array_to_img(batch[0])
            aug_img.save(os.path.join("augmented_images", f"{filename.split('.')[0]}_aug_{i}.png"))
            i += 1
            if i >= 5:
                break

## Displaying the Augmented Images

### Lists to store original and augmented images

In [ ]:
original_images = []
augmented_images = [] 

### Loop through preprocessed images and augment them

In [ ]:
for filename in os.listdir("preprocessed_images"):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.JPG', '.webp')):
        img_path = os.path.join("preprocessed_images", filename)
        
        # Loading image and converting to array format
        img = tf.keras.preprocessing.image.load_img(img_path)
        original_images.append(img) 
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = np.expand_dims(img_array, 0)

        # Creating 5 augmented versions and save each with a unique name
        i = 0
        for batch in datagen.flow(img_array, batch_size=1):
            aug_img = tf.keras.preprocessing.image.array_to_img(batch[0])
            aug_img.save(os.path.join("augmented_images", f"{filename.split('.')[0]}_aug_{i}.png"))
            augmented_images.append(aug_img)  
            i += 1
            if i >= 5:
                break

### Specifying the number of images to display

In [ ]:
num_images_to_display = min(len(original_images), len(augmented_images) // 5)

### Calculating the total number of images to display (original + augmented)

In [ ]:
total_images_to_display = num_images_to_display + (num_images_to_display * 5)

### Calculating the grid size needed for displaying the images

In [ ]:
rows = num_images_to_display 

### Displaying the original and augmented images

In [ ]:
plt.figure(figsize=(15, rows * 3)) 
plt.suptitle("Augmented Images", fontsize=20)  

for i in range(num_images_to_display):
    # Display original image
    plt.subplot(rows, 6, i * 6 + 1) 
    plt.imshow(original_images[i])
    plt.axis('off')
    plt.title("Original", fontsize=14)

    # Display augmented images
    for j in range(5):
        plt.subplot(rows, 6, i * 6 + j + 2)  
        plt.imshow(augmented_images[i * 5 + j])
        plt.axis('off')
        plt.title(f"Aug {j + 1}", fontsize=14)

plt.tight_layout()
plt.subplots_adjust(top=0.976)
plt.show()